In [ ]:
import torchvision.datasets as datasets
# Datanı yükləyir
train_data = datasets.CIFAR10(root='./data', train=True, download=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# SEED SETUP
SEED = 20240208  # <--- Müəllimin verdiyi seed-i buraya yaz
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cihaz: {device}")

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Datanı yükləyirik
train_set = torchvision.datasets.CIFAR10(root='/kaggle/working/data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='/kaggle/working/data', train=False, download=True, transform=transform)

# Nəqliyyat sinifləri: 0: airplane, 1: automobile, 8: ship, 9: truck
vehicle_classes = [0, 1, 8, 9]
class_map = {0: 0, 1: 1, 8: 2, 9: 3}
inv_class_map = {0: 'airplane', 1: 'automobile', 2: 'ship', 3: 'truck'}

def get_filtered_indices(dataset):
    return [i for i, label in enumerate(dataset.targets) if label in vehicle_classes]

# Filterləmə prosesi
train_idx = get_filtered_indices(train_set)
test_idx = get_filtered_indices(test_set)

# Etiketləri 0-3 aralığına gətiririk
for i in train_idx: train_set.targets[i] = class_map[train_set.targets[i]]
for i in test_idx: test_set.targets[i] = class_map[test_set.targets[i]]

train_loader = DataLoader(Subset(train_set, train_idx), batch_size=64, shuffle=True)
test_loader = DataLoader(Subset(test_set, test_idx), batch_size=64, shuffle=False)

In [ ]:
class VehicleCNN(nn.Module):
    def __init__(self):
        super(VehicleCNN, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 4)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        return self.classifier(x)

In [ ]:
def train_model(version_name, opt_type, lr):
    model = VehicleCNN().to(device)
    criterion = nn.CrossEntropyLoss()

    if opt_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    else: # Option C: SGD with momentum
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    history = {'loss': [], 'acc': []}

    for epoch in range(20):
        model.train()
        correct, total, running_loss = 0, 0, 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        acc = 100. * correct / total
        history['loss'].append(running_loss/len(train_loader))
        history['acc'].append(acc)
        print(f"{version_name} - Epoch {epoch+1}/20 | Loss: {history['loss'][-1]:.3f} | Acc: {acc:.2f}%")

    return model, history

In [ ]:
print("--- Version 1: Adam Başlayır ---")
model1, hist1 = train_model("V1 (Adam)", "Adam", 0.001)

print("\n--- Version 2: SGD Başlayır ---")
model2, hist2 = train_model("V2 (SGD)", "SGD", 0.001)

# Müqayisə Cədvəli və Qrafiklər
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(hist1['acc'], label='V1 (Adam)')
plt.plot(hist2['acc'], label='V2 (SGD)')
plt.title('Training Accuracy Comparison')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist1['loss'], label='V1 (Adam)')
plt.plot(hist2['loss'], label='V2 (SGD)')
plt.title('Training Loss Comparison')
plt.legend()
plt.show()

In [ ]:
model1.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        outputs = model1(imgs.to(device))
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

# Matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Airplane', 'Auto', 'Ship', 'Truck'])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix (Best Model)")
plt.show()

# 10 Samples
imgs, labels = next(iter(test_loader))
outputs = model1(imgs[:10].to(device))
_, preds = torch.max(outputs, 1)

plt.figure(figsize=(15, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    img = imgs[i].permute(1, 2, 0).numpy()
    img = (img * 0.2 + 0.5).clip(0, 1) # Unnormalize for display
    plt.imshow(img)
    plt.title(f"Real: {inv_class_map[labels[i].item()]}\nPred: {inv_class_map[preds[i].item()]}")
    plt.axis('off')
plt.show()